In [15]:
%load_ext autoreload
%autoreload 2
from data import load_and_process_soccer_events, load_and_process_metadata, load_and_process_roosters
from enrich import enrich_roosters
from config import TEAM_ABBREVS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
event_df, players_df = load_and_process_soccer_events("/home/soccerdata/FIFA_WorldCup_2022/Event Data")
metadata_df = load_and_process_metadata("/home/soccerdata/FIFA_WorldCup_2022/Metadata")

In [ ]:
# 808 unique players, some teams have less than 26 players. 
rooster_df = load_and_process_roosters("/home/soccerdata/FIFA_WorldCup_2022/Rosters")
rooster_df

In [ ]:
rooster_df.drop("playerNickname")
rooster_df

In [ ]:
import polars as pl
teams_to_enrich = list(TEAM_ABBREVS.keys())

Setup and Run Transfermarkt API:

1. git clone git@github.com:felipeall/transfermarkt-api.git
2. cd transfermarkt-api
3. pip install -r transfermarkt-api/requirements.txt
4. uvicorn app.main:app --reload

The API will be available at http://127.0.0.1:8000.

In [ ]:
from enrich import enrich_roosters
enrich_roosters(rooster_df)